# Setup

In [1]:
%pip install -qU IProgress
%pip install -qU ipywidgets
%pip install -qU python-dotenv
%pip install -qU tqdm

%pip install -qU langchain-core
%pip install -qU langchain-community
%pip install -qU langchain-openai 
%pip install -qU langchain-qdrant 

%pip install -qU ragas
%pip install -qU unstructured

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Environment Variables

In [2]:
# Get environment variables

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

# Utilities

In [3]:
# Load docs from a directory

from langchain_community.document_loaders import DirectoryLoader 

def process_directory(
    path: str, 
    glob: str, 
    loader_cls: str = None, 
    use_multithreading: bool = True
) -> None:
    	
	if loader_cls is None:
		loader = DirectoryLoader(path=path, 
						   glob=glob, 
						   show_progress=True, 
						   use_multithreading=use_multithreading)
	else:
		loader = DirectoryLoader(path=path, 
						   glob=glob, 
						   loader_cls=loader_cls, 
						   show_progress=True, 
						   use_multithreading=use_multithreading)
	
	docs = loader.load()
	
	return docs

#####

def test_process_directory():
	docs = process_directory(path="docs/10k/html", 
						  glob="**/*.html", 
						  loader_cls=None, 
						  use_multithreading=True)
	print(len(docs))

test_process_directory()

  0%|          | 0/2 [00:00<?, ?it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
100%|██████████| 2/2 [00:03<00:00,  1.61s/it]

2


In [4]:
# Create embeddings using OpenAI

from langchain_openai import OpenAIEmbeddings

def create_embeddings_openai(model="text-embedding-ada-002") -> OpenAIEmbeddings:

	# Initialize the OpenAIEmbeddings class
	embeddings = OpenAIEmbeddings(model=model)

	return embeddings

#####

def test_create_embeddings_openai():
	text = "What is the annual revenue of Uber?"
	
	embeddings = create_embeddings_openai()
	vector = embeddings.embed_query(text)
	
	print(vector)

test_create_embeddings_openai()

[0.020656345412135124, -0.04052777588367462, 0.02295876294374466, -0.02543124556541443, -0.010001124814152718, -0.023979153484106064, -0.021938374266028404, -0.01661403477191925, -0.02121886797249317, -0.02864939719438553, 0.02060401812195778, 0.007528643123805523, -0.007450151722878218, -0.013232358731329441, 0.0011029690504074097, -0.0037643215619027615, 0.012774491682648659, -0.010445910505950451, 0.017660588026046753, 0.015724463388323784, -0.0030971437226980925, -0.0004223006253596395, -0.04633614793419838, 0.006590015720576048, 0.0023890850134193897, 0.029198838397860527, 0.024685576558113098, 0.0007219582912512124, 0.025025706738233566, -0.02121886797249317, 0.008274312131106853, 0.004097910597920418, -0.025391999632120132, 0.02114037796854973, -0.036184582859277725, -0.0036825595889240503, 0.008235066197812557, 0.0011201391462236643, 0.03835618123412132, 0.02650396339595318, 0.01725504733622074, 0.0068156784400343895, -0.009726405143737793, -0.005200061947107315, -0.01828851923

In [5]:
# Create a text splitter using recursive character text splitter

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/

from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_docs_recursive(docs: list, 
						 chunk_size=500, 
						 chunk_overlap=50
						 ) -> list:

	text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, 
												chunk_overlap=chunk_overlap)

	chunks = text_splitter.split_documents(docs)

	return chunks

#####

def test_chunk_docs_recursive(): 
	docs = process_directory(path="docs/10k/html", 
						  glob="**/*.html", 
						  loader_cls=None, 
						  use_multithreading=True)
	
	chunks = chunk_docs_recursive(docs=docs)

	print(f"\nNumber of chunks = {len(chunks)}\n")

	print(f"First chunk = {chunks[0].page_content}")

test_chunk_docs_recursive()

  0%|          | 0/2 [00:00<?, ?it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
100%|██████████| 2/2 [00:01<00:00,  1.14it/s]


Number of chunks = 2344

First chunk = UNITED STATES

SECURITIES AND EXCHANGE COMMISSION

Washington, D.C. 20549

FORM

(Mark One)

☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

For the fiscal year ended

OR

☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 FOR THE TRANSITION PERIOD FROM                      TO

Commission File Number

(Exact name of registrant as specified in its charter)


In [10]:
# Create a Qdrant vector store

from langchain_core.embeddings import Embeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

def create_qdrant_vector_store(location: str, 
							   collection_name: str, 
							   vector_size: int, 
							   embeddings: Embeddings, 
							   docs: list) -> QdrantVectorStore:

	# Initialize the Qdrant client
	qdrant_client = QdrantClient(location=location)

	# Create a collection in Qdrant
	qdrant_client.create_collection(collection_name=collection_name, 
								 vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE))

	# Initialize QdrantVectorStore with the Qdrant client
	qdrant_vector_store = QdrantVectorStore(client=qdrant_client, 
										 collection_name=collection_name, embedding=embeddings)
	
	# Add the docs to the vector store
	qdrant_vector_store.add_documents(docs)
	
	return qdrant_vector_store

#####

def test_create_qdrant_vector_store():
	docs = process_directory(path="docs/10k/html", 
						  glob="**/*.html", 
						  loader_cls=None, 
						  use_multithreading=True)
	
	chunks = chunk_docs_recursive(docs=docs)
	
	embeddings = create_embeddings_openai()

	vector_store = create_qdrant_vector_store(":memory:", 
										   "holiday-test", 
										   1536, 
										   embeddings, 
										   chunks)
	
	print(vector_store.collection_name)

test_create_qdrant_vector_store()

  0%|          | 0/2 [00:00<?, ?it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
100%|██████████| 2/2 [00:01<00:00,  1.08it/s]


holiday-test


In [11]:
# Create a Qdrant retriever

from langchain_core.retrievers import BaseRetriever
from langchain_qdrant import QdrantVectorStore

def create_retriever_qdrant(vector_store: QdrantVectorStore) -> BaseRetriever:

	retriever = vector_store.as_retriever()

	return retriever

#####

def test_create_retriever_qdrant(text):
	docs = process_directory(path="docs/10k/html", 
						  glob="**/*.html", 
						  loader_cls=None, 
						  use_multithreading=True)
	
	chunks = chunk_docs_recursive(docs=docs)
	
	embeddings = create_embeddings_openai()

	vector_store = create_qdrant_vector_store(":memory:", 
										   "holiday-test", 
										   1536, 
										   embeddings, 
										   chunks)
	
	retriever = create_retriever_qdrant(vector_store)
	
	docs = retriever.invoke(text)

	print(docs[0])

test_create_retriever_qdrant("What is the annual revenue for Uber?")

  0%|          | 0/2 [00:00<?, ?it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
100%|██████████| 2/2 [00:01<00:00,  1.16it/s]


page_content='Net income attributable to Uber Technologies, Inc. was $1.9 billion, which includes the favorable impact of a pre-tax unrealized gain on debt and equity securities, net, of $1.6 billion primarily related to changes in the fair value of our equity securities, including: a $985 million net unrealized gain on our Aurora investment, a $443 million net unrealized gain on our Didi investment, a $84 million net unrealized gain on our Joby investment, and a $80 million net unrealized gain on our Grab' metadata={'source': 'docs/10k/html/uber-20231231.html', '_id': '303484c10f6a4bd692e47f575af0713a', '_collection_name': 'holiday-test'}


In [12]:
# Create a prompt template

# https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/#chatprompttemplate
# https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

from langchain.prompts import ChatPromptTemplate

def create_chat_prompt_template(template: str = None) -> ChatPromptTemplate:
	
	if template is None:
		template = '''
		You are an expert assistant designed to help users analyze and answer questions about 10K annual reports filed by publicly traded companies. Users may ask about specific sections, financial metrics, trends, or comparisons across multiple reports. Your role is to provide accurate, concise, and relevant answers, referencing appropriate sections or data points where applicable.

		When responding, adhere to the following principles:

		Understand the Question: Identify whether the query is focused on a specific company, year, or metric, or if it spans multiple reports for comparison.
		Clarify Uncertainty: If a user's question is unclear, ask for clarification or additional context.
		Locate and Reference Information: Use relevant sections of the 10K report(s), such as MD&A, Financial Statements, Risk Factors, or Notes to Financial Statements, to back up your answers.
		Synthesize Data: Provide summaries or insights when the question involves comparing data or trends across multiple reports.
		Stay Objective: Avoid providing subjective opinions or interpretations beyond the factual content in the reports.
		Example User Queries and Expected Responses:

		"What was the revenue for Company X in 2022 and 2023?"

		Locate and report the revenue figures from the Income Statements of the respective 10K reports for 2022 and 2023.
		"What are the main risk factors for Company Y in its latest report?"

		Summarize the key risk factors from the most recent 10K report's "Risk Factors" section.
		"How did the operating income of Company Z change over the last three years?"

		Extract operating income figures from the 10K reports for the past three years and provide a brief comparison.
		"Compare the debt levels of Company A and Company B in 2023."

		Retrieve debt-related figures from the Balance Sheets or Notes to Financial Statements of both companies and summarize the comparison.
		"What trends are evident in Company W's R&D expenses over the last five years?"

		Summarize trends using data from the Income Statements or footnotes for R&D expenses across five consecutive 10K reports.
		Assumptions and Constraints:

		Only use inforomation from 10K reports provided in the context below. 
		For complex queries spanning multiple reports, provide a structured summary highlighting key comparisons or trends.
		If certain information is unavailable, state so clearly and suggest alternative approaches to obtain it.

		Now it's your turn!
		
		{question}

		{context}
		'''
	
	prompt = ChatPromptTemplate.from_template(template)

	return prompt

#####

def test_create_chat_prompt_template():
	prompt = create_chat_prompt_template()
	
	print(prompt)

test_create_chat_prompt_template()

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n\t\tYou are an expert assistant designed to help users analyze and answer questions about 10K annual reports filed by publicly traded companies. Users may ask about specific sections, financial metrics, trends, or comparisons across multiple reports. Your role is to provide accurate, concise, and relevant answers, referencing appropriate sections or data points where applicable.\n\n\t\tWhen responding, adhere to the following principles:\n\n\t\tUnderstand the Question: Identify whether the query is focused on a specific company, year, or metric, or if it spans multiple reports for comparison.\n\t\tClarify Uncertainty: If a user\'s question is unclear, ask for clarification or additional context.\n\t\tLocate and Reference Information: Use relevant sections of the 1

In [13]:
# Create a Langchain chain

from operator import itemgetter

from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import RunnablePassthrough, RunnableSerializable
from langchain_openai import ChatOpenAI

def create_chain (model: str, 
				  prompt_template: ChatPromptTemplate, 
				  retriever: BaseRetriever
				  ) -> RunnableSerializable:

	llm = ChatOpenAI(model=model)
		
	chain = (
		{"context": itemgetter("question") | retriever, "question": itemgetter("question")} 
		| RunnablePassthrough.assign(context=itemgetter("context")) 
		| {"response": prompt_template | llm, "context": itemgetter("context")}
		)

	return chain

#####

def test_create_chain_qdrant():
	docs = process_directory(path="docs/10k/html", 
						  glob="**/*.html", 
						  loader_cls=None, 
						  use_multithreading=True)

	chunks = chunk_docs_recursive(docs=docs)

	embeddings = create_embeddings_openai()

	vector_store = create_qdrant_vector_store(":memory:", 
										   "holiday-test", 
										   1536, 
										   embeddings, 
										   chunks)

	retriever = create_retriever_qdrant(vector_store)

	chat_prompt_template = create_chat_prompt_template()

	chain = create_chain("gpt-4o-mini", 
					  chat_prompt_template, 
					  retriever)
	
	result = chain.invoke({"question" : "What is the annual revenue of Uber?"})
	
	print(result)

test_create_chain_qdrant()

  0%|          | 0/2 [00:00<?, ?it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
100%|██████████| 2/2 [00:01<00:00,  1.13it/s]


{'response': AIMessage(content="The annual revenue of Uber Technologies, Inc. for the year ended December 31, 2023, is $37,281 million (or $37.3 billion). This represents a 17% increase from the previous year's revenue of $31,877 million (or $31.9 billion) for the year ended December 31, 2022.\n\nThis information can be found in the Income Statement section of Uber's 10K report.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 1364, 'total_tokens': 1456, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-2a689992-5a9f-49f4-a197-9785438ba2b8-0', usage_metadata={'input_tokens': 1364, 'output_tokens': 92, 'total_tokens': 1456, 'input_token_de

In [15]:
# Generate answers from a chain using a list of questions

from typing import List, Tuple

from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.runnables import RunnableSerializable


def generate_answers_contexts(chain: RunnableSerializable, 
							  questions: list
							  ) -> Tuple[List, List]:
	
	answers = []
	contexts = []

	# Loop over the list of questions and call the chain to get the answer and context
	for question in questions:
		print(question)

		# Call the chain to get answers and contexts
		response = chain.invoke({"question" : question})
		print(response)

		# Capture the answer and context 
		answers.append(response["response"].content)
		contexts.append([context.page_content for context in response["context"]])

	return answers, contexts

#####

def test_generate_answers_contexts():
	docs = process_directory(path="docs/10k/html", 
						  glob="**/*.html", 
						  loader_cls=None, 
						  use_multithreading=True)
	
	chunks = chunk_docs_recursive(docs=docs)

	embeddings = create_embeddings_openai()

	vector_store = create_qdrant_vector_store(":memory:", 
										   "holiday-test", 
										   1536, 
										   embeddings, 
										   chunks)
	
	retriever = create_retriever_qdrant(vector_store)

	chat_prompt_template = create_chat_prompt_template()

	chain = create_chain("gpt-4o-mini", 
					  chat_prompt_template, 
					  retriever)
	
	questions = ["What is the annual revenue of Lyft?",
			  "What is the annual revenue of Uber?",
			  "Which company has a larger annual revenue - Lyft or Uber?"]	
	
	answers, contexts = generate_answers_contexts(chain=chain, 
											   questions=questions)
	
	print(f"Total number of answers = {len(answers)}")
	print(f"Total number of contexts = {len(contexts)}")

test_generate_answers_contexts()

  0%|          | 0/2 [00:00<?, ?it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
100%|██████████| 2/2 [00:01<00:00,  1.13it/s]


What is the annual revenue of Lyft?
{'response': AIMessage(content='The annual revenue of Lyft for the fiscal year ended December 31, 2023, can be found in the Consolidated Statements of Operations section of their 10K report. However, the provided excerpts do not include specific numerical figures for the revenue for that year. \n\nTo accurately answer your question, I would need to reference the financial statements from the report that contain the actual revenue figures. If you have access to the full financial statements, you can look there, or I can assist you in finding a specific section if more detailed information or context is provided.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 114, 'prompt_tokens': 1032, 'total_tokens': 1146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens':

In [19]:
# Run a Ragas evaluation 

import time
from typing import Tuple

import pandas as pd
from pandas import DataFrame

from datasets import Dataset
from langchain_core.runnables import RunnableSerializable
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness,
)

def run_ragas_evaluation(chain: RunnableSerializable, 
						questions: list, 
						groundtruths: list, 
						eval_metrics: list = [answer_correctness, answer_relevancy, context_recall, context_precision, faithfulness]
						):
	
	answers = []
	contexts = []
	answers, contexts = generate_answers_contexts(chain=chain, 
                                               questions=questions)

	# Create the input dataset 
	input_dataset = Dataset.from_dict({"question" : questions,       	# From the dataframe
										"answer" : answers,             # From the chain
										"contexts" : contexts,          # From the chain
										"ground_truth" : groundtruths   # From the dataframe
										})

	# Run the Ragas evaluation using the input dataset and eval metrics
	ragas_results = evaluate(input_dataset, 
                          eval_metrics)
      
	ragas_results_df = ragas_results.to_pandas()
	
	return ragas_results, ragas_results_df
	
#####

def test_run_ragas_evaluation():
    docs = process_directory(
        path="docs/10k/html",
        glob="**/*.html",
        loader_cls=None,
        use_multithreading=True
    )

    chunks = chunk_docs_recursive(docs=docs)

    embeddings = create_embeddings_openai()

    vector_store = create_qdrant_vector_store(
        ":memory:",
        "holiday-test",
        1536,
        embeddings,
        chunks
    )

    retriever = create_retriever_qdrant(vector_store)

    chat_prompt_template = create_chat_prompt_template()

    chain = create_chain("gpt-4o-mini", chat_prompt_template, retriever)

    testset_df = pd.read_csv("testsets/10k_testset.csv")

    questions = testset_df["user_input"].values.tolist()
    questions = [str(question) for question in questions]
    top_5_questions = questions[:5]

    groundtruths = testset_df["reference"].values.tolist()
    groundtruths = [str(ground_truth) for ground_truth in groundtruths]
    top_5_groundtruths = groundtruths[:5]

    eval_metrics = [
        answer_correctness,
        answer_relevancy,
        context_precision,
        context_recall,
        faithfulness
    ]
    
    ragas_results, ragas_results_df = run_ragas_evaluation(chain, 
                                                           	top_5_questions, 
                                                        	top_5_groundtruths, 
                                                        	eval_metrics)

    timestr = time.strftime("%Y%m%d%H%M%S")
    ragas_results_df.to_csv(f"evaluations/10x_test_testset_evaluation_{timestr}.csv")

    print(ragas_results)

test_run_ragas_evaluation()

  0%|          | 0/2 [00:00<?, ?it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
100%|██████████| 2/2 [00:01<00:00,  1.15it/s]


What kind of information does the Form 10-K provide about a company's future performance and risks?
{'response': AIMessage(content='The Form 10-K provides important insights regarding a company\'s future performance and associated risks through several key sections:\n\n1. **Forward-Looking Statements**: Companies include forward-looking statements that relate to potential future events or performance. These statements contain words like "may," "will," "expect," "plan," and "project," indicating uncertainty about actual outcomes. For example, Lyft and Uber each highlight that these statements involve substantial risks and uncertainties, and historical results may not predict future performance.\n\n2. **Management\'s Discussion and Analysis (MD&A)**: In this section, companies often outline their expectations regarding future operations and financial health. This analysis may include trends, factors affecting performance, and management\'s strategic plans, all of which offer a forward-lo

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

{'answer_correctness': 0.4922, 'answer_relevancy': 0.7811, 'context_precision': 0.7000, 'context_recall': 0.5000, 'faithfulness': 0.8489}


# Test 1a - OpenAI and Qdrant

In [20]:
# Build chain using OpenAI and Qdrant

embeddings = create_embeddings_openai()

docs = process_directory(path="docs/10k/html", 
						  glob="**/*.html", 
						  loader_cls=None, 
						  use_multithreading=True)

chunks = chunk_docs_recursive(docs=docs)

print(f'\nNumber of chunks = {len(chunks)}\n')

vector_store = create_qdrant_vector_store(':memory:', 
                                          'holiday-test', 
                                          1536, 
                                          embeddings, 
                                          chunks)

retriever = create_retriever_qdrant(vector_store)

chat_prompt_template = create_chat_prompt_template()

chain = create_chain('gpt-4o', 
                     chat_prompt_template, 
                     retriever)

  0%|          | 0/2 [00:00<?, ?it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
100%|██████████| 2/2 [00:01<00:00,  1.13it/s]



Number of chunks = 2344



In [21]:
# Test the chain with a few questions 

questions = ["What is the annual revenue of Uber?",
"What is the annual revenue of Lyft?",
"How does Uber's revenue compare to Lyft's revenue?",]

for question in questions:
	print(question)
	result = chain.invoke({"question" : question})
	print(result)
	print(result["response"].content)
	print("\n*****")

What is the annual revenue of Uber?
{'response': AIMessage(content='The annual revenue of Uber for the years 2022 and 2023 are as follows:\n\n- **2022:** $31,877 million\n- **2023:** $37,281 million\n\nThese figures are sourced from the section of the 10K report specifying the revenue for Uber in the respective years.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 1369, 'total_tokens': 1434, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-10cd1762-5e41-46c7-89e0-6f22adb81c06-0', usage_metadata={'input_tokens': 1369, 'output_tokens': 65, 'total_tokens': 1434, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'au

In [22]:
# Evaluate the chain using Ragas

import time

import pandas as pd

# Get the questions and groundtruths from the dataframe
testset_df = pd.read_csv("testsets/10k_testset.csv")

questions = testset_df["user_input"].values.tolist()
questions = [str(question) for question in questions]

groundtruths = testset_df["reference"].values.tolist()
groundtruths = [str(ground_truth) for ground_truth in groundtruths]

eval_metrics = [
	answer_correctness,
	answer_relevancy,
	context_precision,
	context_recall,
	faithfulness
]

ragas_results, ragas_results_df = run_ragas_evaluation(chain, 
														questions, 
														groundtruths, 
														eval_metrics)

# Write the results to disk
timestr = time.strftime("%Y%m%d%H%M%S")
ragas_results_df.to_csv(f"evaluations/10x_test1_testset_evaluation_{timestr}.csv")

# Show the resutls
print(ragas_results)

What kind of information does the Form 10-K provide about a company's future performance and risks?
{'response': AIMessage(content='The Form 10-K provides information about a company\'s future performance and risks primarily through the inclusion of "forward-looking statements" and the "Risk Factors" section.\n\n1. **Forward-Looking Statements:**\n   - The 10-K reports from both Lyft and Uber mention that they contain forward-looking statements. These statements typically relate to future events or the company\'s future financial or operating performance. They involve substantial risks and uncertainties and can be identified by words like “may,” “will,” “should,” “expect,” “plan,” “anticipate,” “could,” “intend,” “target,” and “project”.\n   - Forward-looking statements are meant to give investors insights into management\'s expectations and projections regarding future operations, financial conditions, and strategies. However, they are not guarantees but are subject to change based on

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

{'answer_correctness': 0.4984, 'answer_relevancy': 0.7927, 'context_precision': 0.7000, 'context_recall': 0.5000, 'faithfulness': 0.9165}
